# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9dd64f29d0>
├── 'a' --> tensor([[-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890]])
└── 'x' --> <FastTreeValue 0x7f9dd64f2b80>
    └── 'c' --> tensor([[ 0.0284,  0.0632,  1.1628,  1.2552],
                        [ 0.1224, -0.4259,  1.9420, -0.4451],
                        [ 0.1487,  1.7391, -0.0879, -0.2843]])

In [4]:
t.a

tensor([[-2.2093,  0.9755, -1.5850],
        [ 0.4887,  1.0629, -0.5890]])

In [5]:
%timeit t.a

63.8 ns ± 0.0702 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9dd64f29d0>
├── 'a' --> tensor([[-0.3334,  1.2884, -0.3426],
│                   [ 0.9516,  0.1538,  0.9026]])
└── 'x' --> <FastTreeValue 0x7f9dd64f2b80>
    └── 'c' --> tensor([[ 0.0284,  0.0632,  1.1628,  1.2552],
                        [ 0.1224, -0.4259,  1.9420, -0.4451],
                        [ 0.1487,  1.7391, -0.0879, -0.2843]])

In [7]:
%timeit t.a = new_value

66.2 ns ± 0.208 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.2093,  0.9755, -1.5850],
               [ 0.4887,  1.0629, -0.5890]]),
    x: Batch(
           c: tensor([[ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843]]),
       ),
)

In [10]:
b.a

tensor([[-2.2093,  0.9755, -1.5850],
        [ 0.4887,  1.0629, -0.5890]])

In [11]:
%timeit b.a

58.1 ns ± 0.044 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3829, -1.1096,  1.4416],
               [-0.4802, -1.6285, -0.5414]]),
    x: Batch(
           c: tensor([[ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.0995 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

819 ns ± 4.08 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11 µs ± 4.47 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 132 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

145 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9d2710d700>
├── 'a' --> tensor([[[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]],
│           
│                   [[-2.2093,  0.9755, -1.5850],
│                    [ 0.4887,  1.0629, -0.5890]]])
└── 'x' --> <FastTreeValue 0x7f9d2710deb0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 60.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9dd445a4f0>
├── 'a' --> tensor([[-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890],
│                   [-2.2093,  0.9755, -1.5850],
│                   [ 0.4887,  1.0629, -0.5890]])
└── 'x' --> <FastTreeValue 0x7f9d2710dc70>
    └── 'c' --> tensor([[ 0.0284,  0.0632,  1.1628,  1.2552],
                        [ 0.1224, -0.4259,  1.9420, -0.4451],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 55.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 74.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0284,  0.0632,  1.1628,  1.2552],
                       [ 0.1224, -0.4259,  1.9420, -0.4451],
                       [ 0.1487,  1.7391, -0.0879, -0.2843]],
              
                      [[ 0.0284,  0.0632,  1.1628,  1.2552],
                       [ 0.1224, -0.4259,  1.9420, -0.4451],
                       [ 0.1487,  1.7391, -0.0879, -0.2843]],
              
                      [[ 0.0284,  0.0632,  1.1628,  1.2552],
                       [ 0.1224, -0.4259,  1.9420, -0.4451],
                       [ 0.1487,  1.7391, -0.0879, -0.2843]],
              
                      [[ 0.0284,  0.0632,  1.1628,  1.2552],
                       [ 0.1224, -0.4259,  1.9420, -0.4451],
                       [ 0.1487,  1.7391, -0.0879, -0.2843]],
              
                      [[ 0.0284,  0.0632,  1.1628,  1.2552],
                       [ 0.1224, -0.4259,  1.9420, -0.4451],
                       [ 0.1487,  1.7391, -0.0879, -0.2843]],

In [26]:
%timeit Batch.stack(batches)

143 µs ± 411 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843],
                      [ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843],
                      [ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843],
                      [ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843],
                      [ 0.0284,  0.0632,  1.1628,  1.2552],
                      [ 0.1224, -0.4259,  1.9420, -0.4451],
                      [ 0.1487,  1.7391, -0.0879, -0.2843],
                      [ 0.0284,  0.0632,  1.1628,  1.2552],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 303 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 3.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
